In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# read data from txt files
movies = open('hw8_movies.txt').read().splitlines()
ids = open('hw8_ids.txt').read().splitlines()
ratings_s = np.loadtxt('hw8_ratings.txt', dtype='str')

In [3]:
ratings = np.zeros((258,76))
for i in range(ratings_s.shape[0]):
    for j in range(ratings_s.shape[1]):
        if ratings_s[i][j] == '?': ratings[i][j] -= 1.0
        else: ratings[i][j] += float(ratings_s[i][j])

### a) Sanity check

In [4]:
import pandas as pd
def pop(rating):
    df = pd.DataFrame({'Movies':movies,'popularity ratings':rating})
    df = df.sort_values(by='popularity ratings')
    df = df.reset_index(drop=True)
    return df

In [5]:
popularity = np.sum(ratings == 1,axis = 0) / np.sum(ratings >= 0, axis = 0)
popularity_sorted = np.argsort(popularity)
pop(popularity)

,Movies,popularity ratings
0,The_Last_Airbender,0.211268
1,Fifty_Shades_of_Grey,0.360000
2,I_Feel_Pretty,0.380952
3,Magic_Mike,0.428571
4,Man_of_Steel,0.525773
5,The_Shape_of_Water,0.561644
6,World_War_Z,0.561798
7,Hustlers,0.565217
8,Prometheus,0.584416
9,Fast_Five,0.625000


### b) Likelihood

$P(\{R_j=r_j^{(t)}\}_{j\in\Omega_t}) = \sum_{i=1}^{k}P(\{R_j=r_j^{(t)}\}_{j\in\Omega_t},Z=i)$(marginalization);
<br/>
<br/>$=\sum_{i=1}^{k}P(Z=i)P(\{R_j=r_j^{(t)}\}_{j\in\Omega_t}|Z=i)$(product rule);
<br/>
<br/>$=\sum_{i=1}^{k}P(Z=i)\prod_{j\in\Omega_t}P(R_j=r_j^{(t)}|Z=i)$(conditional independence:d-seperation(2)).

### c) E-step

$P(Z=i|\{R_j=r_j^{(t)}\}_{j\in\Omega_t}) =$ <font color = black size = 4 face=雅黑>$\frac{P(Z=i,\{R_j=r_j^{(t)}\}_{j\in\Omega_t})}{P(\{R_j=r_j^{(t)}\}_{j\in\Omega_t})}$
</font>(product rule);
<br/>
<br/>Numerator $=P(Z=i)P(\{R_j=r_j^{(t)}\}_{j\in\Omega_t}|Z=i)=P(Z=i)\prod_{j\in\Omega_t}P(R_j=r_j^{(t)}|Z=i)$（product rule & conditional independence:d-seperation(2));
<br/>
<br/>$P(Z=i|\{R_j=r_j^{(t)}\}_{j\in\Omega_t})=$ <font color = black size = 4 face=雅黑> $\frac{P(Z=i)\prod_{j\in\Omega_t}P(R_j=r_j^{(t)}|Z=i)}{\sum_{i^{'}=1}^{k}P(Z=i^{'})\prod_{j\in\Omega_t}P(R_j=r_j^{(t)}|Z=i^{'})}$
</font>(using the result of the previous question).

### d) M-step

Assume that $\rho_{it}=P(Z=i|\{R_j=r_j^{(t)}\}_{j\in\Omega_t})$;
<br/>
<br/>$P(Z=i)\gets\frac{1}{T}P(Z=i|\{R_j=r_j^{(t)}\}_{j\in\Omega_t})=\frac{1}{T}\sum_{t=1}^{T}\rho_{it}$;
<br/>
<br/>$P(R_j=1|Z=i)\gets$<font color = black size = 4 face=雅黑>$\frac{\sum_tP(R_j=1,Z=i|\{R_k=r_k^{(t)}\}_{k\in\Omega_t})}{\sum_tP(Z=i|\{R_k=r_k^{(t)}\}_{k\in\Omega_t})}$
</font>;
<br/>
<br/>Numerator $=\sum_{\{t|j\in\Omega_t\}}I(r_j^{(t)},1)P(Z=i|\{R_k=r_k^{(t)}\}_{k\in\Omega_t})+\sum_{\{t|j\notin\Omega_t\}}P(Z=i|\{R_k=r_k^{(t)}\}_{k\in\Omega_t})P(R_j=1|Z=i,\{R_k=r_k^{(t)}\}_{k\in\Omega_t})$ (product rule);
<br/>
<br/>$=\sum_{\{t|j\in\Omega_t\}}I(r_j^{(t)},1)P(Z=i|\{R_k=r_k^{(t)}\}_{k\in\Omega_t})+\sum_{\{t|j\notin\Omega_t\}}P(Z=i|\{R_k=r_k^{(t)}\}_{k\in\Omega_t})P(R_j=1|Z=i)$ (conditional independence);
<br/>
<br/>$P(R_j=1|Z=i)\gets$<font color = black size = 4 face=雅黑>$\frac{\sum_{\{t|j\in\Omega_t\}}I(r_j^{(t)},1)\rho_{it}+\sum_{\{t|j\notin\Omega_t\}}\rho_{it}P(R_j=1|Z=i)}{\sum_{t=1}^{T}\rho_{it}}$. </font>

### e) Implementation

In [6]:
probR = np.loadtxt('hw8_probR_init.txt')
probZ = np.loadtxt('hw8_probZ_init.txt')
mask1 = (ratings == 1) + 0
mask0 = (ratings == 0) + 0
mask_1 = (ratings == -1) + 0

In [7]:
def C_product(pr,pz):
    result = []
    for i in range(ratings.shape[0]):
        temp1 = mask1[i][:,np.newaxis]
        temp0 = mask0[i][:,np.newaxis]
        temp = pr * temp1 + (1-pr) * temp0
        product = np.prod(temp,axis=0,where=(temp > 0))
        result.append(product)
    result = np.array(result)
    return result

In [8]:
def C_product_log(pr,pz):
    return np.exp(mask1.dot(np.log(pr)) + mask0.dot(np.log(1-pr)))

In [9]:
def C_loss(pr,pz):
    product = C_product(pr,pz)
    return 1/ratings.shape[0] * np.sum(np.log(product.dot(pz)))

In [10]:
def E_posterior(pr,pz):
    product = C_product(pr,pz)
    denominator = (product.dot(pz))[:,np.newaxis]
    numerator = product * pz
    return numerator / denominator

In [11]:
def M_update(pr,pz):
    posterior = E_posterior(pr,pz)
    pz_update = 1/ratings.shape[0] * np.sum(posterior,axis = 0)
    denominator = pz_update * ratings.shape[0]
    numerator = mask1.T.dot(posterior) + mask_1.T.dot(posterior) * pr
    pr_update = numerator / denominator
    return pr_update,pz_update

In [12]:
# learning steps
initial = C_loss(probR,probZ)
log_list = [initial]
print('iteration 0\tlog-likelihood  %.4f'%initial)
import math
max_iteration = 256
for i in range(1,max_iteration+1):
    probR_new,probZ_new = M_update(probR,probZ)
    log_likelihood = C_loss(probR_new,probZ_new)
    log_list.append(log_likelihood)
    probR,probZ = probR_new,probZ_new
    if math.log(i, 2).is_integer():
        print('iteration %d\tlog-likelihood  %.4f'%(i,log_likelihood))

iteration 0	log-likelihood  -29.3276
iteration 1	log-likelihood  -18.1393
iteration 2	log-likelihood  -16.1713
iteration 4	log-likelihood  -14.9416
iteration 8	log-likelihood  -14.2107
iteration 16	log-likelihood  -13.8581
iteration 32	log-likelihood  -13.7650
iteration 64	log-likelihood  -13.7400
iteration 128	log-likelihood  -13.7385
iteration 256	log-likelihood  -13.7337


### f) Personal movie recomendation

In [14]:
idx = ids.index('A53303892')
posterior = E_posterior(probR,probZ)[idx][:,np.newaxis]
expected_rating = (probR * mask_1[idx][:,np.newaxis]).dot(posterior)
df = pd.DataFrame({'Movies':movies,'probability':expected_rating.flatten()})
df = df.sort_values(by='probability')
indexNames = df[df['probability'] == 0].index
df.drop(indexNames,inplace=True)
df = df.reset_index(drop=True)

In [46]:
df

,Movies,probability
0,The_Last_Airbender,0.221903
1,Magic_Mike,0.435489
2,I_Feel_Pretty,0.486032
3,Hustlers,0.623019
4,Good_Boys,0.627659
5,Bridemaids,0.698862
6,Pitch_Perfect,0.704697
7,Us,0.712467
8,Once_Upon_a_Time_in_Hollywood,0.721973
9,Pokemon_Detective_Pikachu,0.726561
